# Data Analysis

# Notebook set-up

In [56]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import io
import statistics as stat

sns.set_style('whitegrid')
sns.set_context('notebook')

# Data loading

In [57]:
# Load the data into a dataframe 
#can_master = pd.read_csv('can-1996-2016.csv')
#cmte_master =pd.read_csv('cm-1996-2014.csv') 
#opex=pd.read_csv('opex-2004-2014.csv')
#oth=pd.read_csv('oth-1996-2014.csv')
#exp=pd.read_csv('expresults_rev.csv')
#election_spend=pd.read_csv('catall.csv')
#results = pd.read_csv('candr.csv')
#catsort= pd.read_csv('catsort_rev.csv')
cost_vote = pd.read_csv('combined_all.csv')

In [58]:
cost_vote.head()

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,...,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,ge_winner_indicator,total_votes,totalspend,costpervote
0,H0AK00089,2010,550.00,0.00,0.00,0.00,0.0,785.00,0.00,0.0,...,0,DEM,38673.0,0.0,77606.0,0.0,0,116279.0,220001.06,1.892010
1,H0AK01038,2010,1569.23,88.93,0.00,0.00,0.0,0.00,0.00,0.0,...,0,REP,24709.0,0.0,0.0,0.0,0,24709.0,169777.74,6.871089
2,H0AL00016,2010,19831.34,909.80,81.19,465.00,0.0,3177.55,2192.99,0.0,...,0,DEM,6672.0,0.0,0.0,0.0,0,6672.0,26957.87,4.040448
3,H0AL01030,2010,7054.34,5233.32,668.95,27082.28,1175.0,1295.21,583.75,0.0,...,0,REP,18725.0,0.0,0.0,0.0,0,18725.0,44377.00,2.369933
4,H0AL01030,2012,3480.00,27.72,381.09,7257.71,0.0,0.00,56.58,0.0,...,0,R,3854.0,0.0,0.0,0.0,0,3854.0,11203.10,2.906876


In [59]:
cost_vote['total_votes']=cost_vote['total_votes'].fillna(0)

In [60]:
cost_vote['totalspend']=cost_vote['totalspend'].fillna(0)

In [61]:
cost_vote['incumbent']=cost_vote['incumbent'].fillna(0)

In [62]:
cost_vote['party']=cost_vote['party'].fillna('None')

In [63]:
cost_vote = cost_vote.drop(['costpervote'], axis=1)

In [64]:
cost_vote['costpervote'] = cost_vote['totalspend'] / cost_vote['total_votes']

In [65]:
cost_vote['costpervote']=pd.to_numeric(cost_vote[''],errors='coerce')

nan

In [34]:
print(catsort[pd.to_numeric(catsort['cycle'], errors = 'coerce').isnull()])

NameError: name 'catsort' is not defined

In [180]:
catsort = catsort[pd.notnull(catsort['cycle'])]

In [181]:
catsort['cycle'] = catsort['cycle'].astype(int).astype('str')

In [182]:
catsort.head()

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,total
0,H6FL19038,2006,281886.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H2NY17071,2004,4500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H8TX27023,2008,12900.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H8OH18088,2012,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H0SC01071,2008,83183.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
exp['cycle']=pd.to_numeric(exp['cycle'],errors='coerce')

In [184]:
print(exp[pd.to_numeric(exp['cycle'], errors = 'coerce').isnull()])

       state_abbreviation district incumbent party  primary_votes  \
187                    CA        4       (I)   REP        63731.0   
188                    CA        4       (I)     R       105015.0   
264                    CA       45       NaN     R         8422.0   
282                    CA       48       NaN     D        39385.0   
283                    CA       48       NaN     D        39385.0   
284                    CA       49       (I)     R        56558.0   
285                    CA       49       (I)     R        71329.0   
286                    CA       49       (I)   REP        60447.0   
287                    CA       49       (I)     R        41369.0   
288                    CA       49       (I)   REP        47693.0   
289                    CA       49       (I)     R        66710.0   
1063                   ID        1       (I)     R        48986.0   
1064                   ID        1       (I)     R        48986.0   
1081                   ID        1

In [185]:
exp = exp[pd.notnull(exp['cycle'])]

In [186]:
exp['cycle'] = exp['cycle'].astype(int).astype('str')

In [187]:
exp.head()

,state_abbreviation,district,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,ge_winner_indicator,total_votes,cand_id,cmte_id,exp_sum,category,cycle
0,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,C00347872,1363.96,NaN,2012
1,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,C00347872,1363.96,NaN,2012
2,AL,7,(I),D,58193.0,NaN,183408.0,NaN,W,241601,H0AL07060,C00347872,1363.96,NaN,2012
3,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,C00347872,1204248.81,NaN,2010
4,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,C00347872,1204248.81,NaN,2010


In [188]:
catsort.head()

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,total
0,H6FL19038,2006,281886.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H2NY17071,2004,4500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H8TX27023,2008,12900.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H8OH18088,2012,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H0SC01071,2008,83183.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
#catsort['cycle']=catsort['cycle'].astype('str')

In [190]:
print(catsort.dtypes)

cand_id     object
cycle       object
c1         float64
c2         float64
c3         float64
c4         float64
c5         float64
c6         float64
c7         float64
c8         float64
c9         float64
c10        float64
c11        float64
c12        float64
c13        float64
total      float64
dtype: object


In [191]:
cs1 = catsort

In [192]:
cs1 = cs1.drop(['c2','c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis=1)

In [193]:
cs1.head()

,cand_id,cycle,c1
0,H6FL19038,2006,281886.64
1,H2NY17071,2004,4500.00
2,H8TX27023,2008,12900.00
3,H8OH18088,2012,30.00
4,H0SC01071,2008,83183.76


In [194]:
cs1.set_index(['cand_id','cycle'], inplace=True)

In [195]:
print(cs1)

                         c1
cand_id   cycle            
H6FL19038 2006    281886.64
H2NY17071 2004      4500.00
H8TX27023 2008     12900.00
H8OH18088 2012        30.00
H0SC01071 2008     83183.76
S0CA00199 2010     65563.32
S0MI00056 2014     49863.62
H2GA12113 2012    162924.41
H8NJ03156 2010     48652.69
H6AZ08020 2006     21606.11
H4WV03070 2014      2826.00
H4WI04191 2006       966.06
H8VA10072 2008      4602.00
H2MA01202 2012     13388.58
H6WV03083 2006      1652.00
H8WY00148 2012     68895.65
H8WI01024 2014    356582.98
H6TX09033 2004   1143785.48
H0IN07157 2012     28824.84
H2MI11042 2014      1938.75
H0FL12150 2010     11395.96
H8AL05109 2012     10019.08
H8VA11039 2010      6902.86
H2MN08103 2012     22125.35
H4TX19136 2012      4743.92
S2CA00286 2006       595.15
H0WI07051 2012      1965.96
H0NY02085 2010   1625923.74
H4TN06051 2008     86814.26
H8CA37137 2012    390071.78
...                     ...
H0MA06072 2012          NaN
H0AZ01267 2012          NaN
S4SC00083 2006      

In [196]:
cs1.head()

,,c1
cand_id,cycle,
H6FL19038,2006,281886.64
H2NY17071,2004,4500.00
H8TX27023,2008,12900.00
H8OH18088,2012,30.00
H0SC01071,2008,83183.76


In [199]:
#test
cs1.loc["H0SC01071", '2006']

/Users/adammorris/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app


c1
cand_id   cycle          
H0SC01071 2006   34484.85
          2006        NaN
          2006        NaN
          2006        NaN
          2006        NaN
          2006        NaN
          2006        NaN

In [200]:
agg_df = cs1.sum(level=['cand_id','cycle'])
print(agg_df)

                        c1
cand_id   cycle           
H0AK00089 2010      550.00
          2012         NaN
H0AK01038 2010     1569.23
H0AL00016 2010    19831.34
H0AL01030 2010     7054.34
          2012     3480.00
H0AL02087 2010         NaN
          2012    11727.82
          2014    12132.25
H0AL02103 2010     2197.40
H0AL02111 2010     4897.58
H0AL03184 2010       14.95
H0AL05049 2004         NaN
          2006         NaN
          2008         NaN
          2010     5818.75
          2012    11016.15
          2014     4592.00
H0AL05155 2010    56555.95
          2012         NaN
H0AL05163 2010    54616.04
          2012    54178.00
          2014    15420.65
H0AL05171 2010     8174.82
H0AL05189 2010     1082.20
H0AL05197 2010         NaN
          2012         NaN
H0AL07060 2004         NaN
          2006         NaN
          2008         NaN
...                    ...
S8TN00246 2008         NaN
S8TX00202 2008         NaN
          2010         NaN
S8TX00228 2008         NaN
S

In [203]:
#test
agg_df.loc['H0SC01071','2006']

c1    34484.85
Name: (H0SC01071, 2006), dtype: float64

In [204]:
#cs1 = cs1[pd.notnull(cs1['c1'])]

In [205]:
#cs1 = cs1.drop_duplicates()

In [206]:
#cs1.groupby(['cand_id','cycle'])['c1'].sum()

In [207]:
cs2 = catsort

In [208]:
cs2 = cs2.drop(['c1','c3','c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis =1)

In [209]:
cs2.set_index(['cand_id','cycle'], inplace=True)

In [210]:
print(cs2)

                 c2
cand_id   cycle    
H6FL19038 2006  NaN
H2NY17071 2004  NaN
H8TX27023 2008  NaN
H8OH18088 2012  NaN
H0SC01071 2008  NaN
S0CA00199 2010  NaN
S0MI00056 2014  NaN
H2GA12113 2012  NaN
H8NJ03156 2010  NaN
H6AZ08020 2006  NaN
H4WV03070 2014  NaN
H4WI04191 2006  NaN
H8VA10072 2008  NaN
H2MA01202 2012  NaN
H6WV03083 2006  NaN
H8WY00148 2012  NaN
H8WI01024 2014  NaN
H6TX09033 2004  NaN
H0IN07157 2012  NaN
H2MI11042 2014  NaN
H0FL12150 2010  NaN
H8AL05109 2012  NaN
H8VA11039 2010  NaN
H2MN08103 2012  NaN
H4TX19136 2012  NaN
S2CA00286 2006  NaN
H0WI07051 2012  NaN
H0NY02085 2010  NaN
H4TN06051 2008  NaN
H8CA37137 2012  NaN
...              ..
H0MA06072 2012  NaN
H0AZ01267 2012  NaN
S4SC00083 2006  NaN
H6TX09033 2004  NaN
S8NM00184 2014  NaN
H6OH15104 2014  NaN
H2TX21094 2012  NaN
S8ID00027 2010  NaN
H4PA08074 2006  NaN
S0NY00410 2012  NaN
S0NH00219 2008  NaN
H4TX09095 2008  NaN
H6HI02301 2006  NaN
S2MO00353 2006  NaN
H8OH01043 2008  NaN
H0NY27066 2010  NaN
H2AL05094 2014  NaN


In [211]:
#cs2 = catsort

In [212]:
cs2.head()

,,c2
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [213]:
#cs2 = cs2[pd.notnull(cs2['c2'])]

In [215]:
#test
cs2.loc["H2NY17071", '2004']

/Users/adammorris/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app


c2
cand_id   cycle          
H2NY17071 2004        NaN
          2004        NaN
          2004   12621.75
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN

In [216]:
agg_df = cs2.sum(level=['cand_id','cycle'])
print(agg_df)

                       c2
cand_id   cycle          
H0AK00089 2010        NaN
          2012        NaN
H0AK01038 2010      88.93
H0AL00016 2010     909.80
H0AL01030 2010    5233.32
          2012      27.72
H0AL02087 2010        NaN
          2012        NaN
          2014        NaN
H0AL02103 2010        NaN
H0AL02111 2010        NaN
H0AL03184 2010    1441.91
H0AL05049 2004        NaN
          2006        NaN
          2008        NaN
          2010        NaN
          2012        NaN
          2014        NaN
H0AL05155 2010   15549.94
          2012        NaN
H0AL05163 2010    2343.79
          2012    8154.95
          2014    8540.32
H0AL05171 2010        NaN
H0AL05189 2010        NaN
H0AL05197 2010        NaN
          2012        NaN
H0AL07060 2004        NaN
          2006        NaN
          2008        NaN
...                   ...
S8TN00246 2008        NaN
S8TX00202 2008        NaN
          2010        NaN
S8TX00228 2008        NaN
S8VA00214 2004        NaN
          20

In [ ]:
#cs2.drop_duplicates()

In [217]:
#test
agg_df.loc['H0NC11118','2010']

c2    4715.72
Name: (H0NC11118, 2010), dtype: float64

In [218]:
cs2.head()

,,c2
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [219]:
#test
agg_df.loc['H2NY17071','2004']

c2    12621.75
Name: (H2NY17071, 2004), dtype: float64

In [ ]:
#cs2.groupby(['cand_id','cycle'])['c2'].sum()

In [220]:
cs3 = catsort

In [221]:
cs3 = cs3.drop(['c1', 'c2', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs3 = cs3[pd.notnull(cs3['c3'])]

In [222]:
cs3.head()

,cand_id,cycle,c3
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [223]:
cs3.set_index(['cand_id','cycle'], inplace=True)

In [ ]:
#cs3.groupby(['cand_id','cycle'])['c3'].sum()

In [224]:
cs3.head()

,,c3
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [ ]:
#cs3 = cs3.drop_duplicates()

In [225]:
agg_df = cs3.sum(level=['cand_id','cycle'])
print(agg_df)

                        c3
cand_id   cycle           
H0AK00089 2010         NaN
          2012         NaN
H0AK01038 2010         NaN
H0AL00016 2010       81.19
H0AL01030 2010      668.95
          2012      381.09
H0AL02087 2010         NaN
          2012    16964.09
          2014    16866.78
H0AL02103 2010         NaN
H0AL02111 2010      484.62
H0AL03184 2010         NaN
H0AL05049 2004         NaN
          2006         NaN
          2008         NaN
          2010         NaN
          2012         NaN
          2014         NaN
H0AL05155 2010   266006.98
          2012     4745.35
H0AL05163 2010    71452.12
          2012   121673.90
          2014    91996.02
H0AL05171 2010         NaN
H0AL05189 2010         NaN
H0AL05197 2010         NaN
          2012         NaN
H0AL07060 2004         NaN
          2006         NaN
          2008         NaN
...                    ...
S8TN00246 2008         NaN
S8TX00202 2008         NaN
          2010         NaN
S8TX00228 2008         NaN
S

In [226]:
#test
agg_df.loc['S8VA00214','2006']

c3    6585.79
Name: (S8VA00214, 2006), dtype: float64

In [227]:
cs4 = catsort

In [228]:
cs4 = cs4.drop(['c1', 'c2', 'c3', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs4 = cs4[pd.notnull(cs4['c4'])]

In [ ]:
#cs4.groupby(['cand_id','cycle'])['c4'].sum()

In [ ]:
#cs4 = cs4.drop_duplicates()

In [229]:
cs4.set_index(['cand_id','cycle'], inplace=True)

In [230]:
cs4.head()

,,c4
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [231]:
agg_df = cs4.sum(level=['cand_id','cycle'])
print(agg_df)

                        c4
cand_id   cycle           
H0AK00089 2010         NaN
          2012         NaN
H0AK01038 2010         NaN
H0AL00016 2010      465.00
H0AL01030 2010    27082.28
          2012     7257.71
H0AL02087 2010         NaN
          2012         NaN
          2014         NaN
H0AL02103 2010         NaN
H0AL02111 2010     8403.45
H0AL03184 2010     5000.00
H0AL05049 2004         NaN
          2006         NaN
          2008         NaN
          2010         NaN
          2012         NaN
          2014         NaN
H0AL05155 2010    20809.92
          2012         NaN
H0AL05163 2010   547321.04
          2012   165131.27
          2014    81481.66
H0AL05171 2010    40231.96
H0AL05189 2010         NaN
H0AL05197 2010      595.35
          2012         NaN
H0AL07060 2004         NaN
          2006         NaN
          2008         NaN
...                    ...
S8TN00246 2008         NaN
S8TX00202 2008         NaN
          2010         NaN
S8TX00228 2008         NaN
S

In [232]:
cs5 = catsort

In [233]:
cs5 = cs5.drop(['c1', 'c2', 'c3', 'c4', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs5 = cs5[pd.notnull(cs5['c5'])]

In [ ]:
#cs5.groupby(['cand_id','cycle'])['c5'].sum()

In [ ]:
#cs5 = cs5.drop_duplicates()

In [234]:
cs5.set_index(['cand_id','cycle'], inplace=True)

In [235]:
cs5.head()

,,c5
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [236]:
agg_df = cs5.sum(level=['cand_id','cycle'])
print(agg_df)

                      c5
cand_id   cycle         
H0AK00089 2010       NaN
          2012       NaN
H0AK01038 2010       NaN
H0AL00016 2010       NaN
H0AL01030 2010    1175.0
          2012       NaN
H0AL02087 2010       NaN
          2012       NaN
          2014       NaN
H0AL02103 2010       NaN
H0AL02111 2010       NaN
H0AL03184 2010       NaN
H0AL05049 2004       NaN
          2006       NaN
          2008       NaN
          2010       NaN
          2012       NaN
          2014       NaN
H0AL05155 2010       NaN
          2012       NaN
H0AL05163 2010   27000.0
          2012   17000.0
          2014   15000.0
H0AL05171 2010       NaN
H0AL05189 2010       NaN
H0AL05197 2010       NaN
          2012       NaN
H0AL07060 2004       NaN
          2006       NaN
          2008       NaN
...                  ...
S8TN00246 2008       NaN
S8TX00202 2008       NaN
          2010       NaN
S8TX00228 2008       NaN
S8VA00214 2004       NaN
          2006       NaN
          2008       NaN


In [237]:
#test
agg_df.loc['H0CA02138','2010']

c5    16750.0
Name: (H0CA02138, 2010), dtype: float64

In [238]:
cs5.head()

,,c5
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [239]:
cs6 = catsort

In [240]:
cs6 = cs6.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs6 = cs6[pd.notnull(cs6['c6'])]

In [ ]:
#cs6.groupby(['cand_id','cycle'])['c6'].sum()

In [ ]:
#cs6 = cs6.drop_duplicates()

In [241]:
cs6.set_index(['cand_id','cycle'], inplace=True)

In [242]:
cs6.head()

,,c6
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [243]:
agg_df = cs6.sum(level=['cand_id','cycle'])
print(agg_df)

                       c6
cand_id   cycle          
H0AK00089 2010     785.00
          2012        NaN
H0AK01038 2010        NaN
H0AL00016 2010    3177.55
H0AL01030 2010    1295.21
          2012        NaN
H0AL02087 2010        NaN
          2012        NaN
          2014        NaN
H0AL02103 2010        NaN
H0AL02111 2010    7489.74
H0AL03184 2010    2293.50
H0AL05049 2004        NaN
          2006        NaN
          2008        NaN
          2010        NaN
          2012        NaN
          2014        NaN
H0AL05155 2010        NaN
          2012        NaN
H0AL05163 2010   66857.04
          2012   16138.18
          2014   11833.66
H0AL05171 2010    5993.05
H0AL05189 2010        NaN
H0AL05197 2010        NaN
          2012        NaN
H0AL07060 2004        NaN
          2006        NaN
          2008        NaN
...                   ...
S8TN00246 2008        NaN
S8TX00202 2008        NaN
          2010        NaN
S8TX00228 2008        NaN
S8VA00214 2004        NaN
          20

In [244]:
cs6.head()

,,c6
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [245]:
#test
agg_df.loc['H0AR01109','2010']

c6    6671.16
Name: (H0AR01109, 2010), dtype: float64

In [246]:
cs7 = catsort

In [247]:
cs7 = cs7.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs7 = cs7[pd.notnull(cs7['c7'])]

In [ ]:
#cs7.groupby(['cand_id','cycle'])['c7'].sum()

In [ ]:
#cs7 = cs7.drop_duplicates()

In [248]:
cs7.set_index(['cand_id','cycle'], inplace=True)

In [249]:
cs7.head()

,,c7
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [250]:
agg_df = cs7.sum(level=['cand_id','cycle'])
print(agg_df)

                       c7
cand_id   cycle          
H0AK00089 2010        NaN
          2012        NaN
H0AK01038 2010        NaN
H0AL00016 2010    2192.99
H0AL01030 2010     583.75
          2012      56.58
H0AL02087 2010        NaN
          2012        NaN
          2014        NaN
H0AL02103 2010        NaN
H0AL02111 2010        NaN
H0AL03184 2010        NaN
H0AL05049 2004        NaN
          2006        NaN
          2008        NaN
          2010        NaN
          2012        NaN
          2014        NaN
H0AL05155 2010   47101.73
          2012        NaN
H0AL05163 2010    9362.66
          2012    9083.92
          2014    2350.00
H0AL05171 2010        NaN
H0AL05189 2010     205.00
H0AL05197 2010        NaN
          2012        NaN
H0AL07060 2004        NaN
          2006        NaN
          2008        NaN
...                   ...
S8TN00246 2008        NaN
S8TX00202 2008        NaN
          2010        NaN
S8TX00228 2008        NaN
S8VA00214 2004        NaN
          20

In [251]:
#test
agg_df.loc['H0AL02087','2010']

c7   NaN
Name: (H0AL02087, 2010), dtype: float64

In [252]:
cs8 = catsort

In [253]:
cs8 = cs8.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs8 = cs8[pd.notnull(cs8['c8'])]

In [ ]:
#cs8.groupby(['cand_id','cycle'])['c8'].sum()

In [ ]:
#cs8 = cs8.drop_duplicates()

In [254]:
cs8.head()

,cand_id,cycle,c8
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [255]:
cs8.set_index(['cand_id','cycle'], inplace=True)

In [256]:
cs8.head()

,,c8
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [257]:
agg_df = cs8.sum(level=['cand_id','cycle'])
print(agg_df)

                    c8
cand_id   cycle       
H0AK00089 2010     NaN
          2012     NaN
H0AK01038 2010     NaN
H0AL00016 2010     NaN
H0AL01030 2010     NaN
          2012     NaN
H0AL02087 2010     NaN
          2012     NaN
          2014     NaN
H0AL02103 2010     NaN
H0AL02111 2010     NaN
H0AL03184 2010     NaN
H0AL05049 2004     NaN
          2006     NaN
          2008     NaN
          2010     NaN
          2012     NaN
          2014     NaN
H0AL05155 2010     NaN
          2012     NaN
H0AL05163 2010   249.0
          2012     NaN
          2014     NaN
H0AL05171 2010     NaN
H0AL05189 2010     NaN
H0AL05197 2010     NaN
          2012     NaN
H0AL07060 2004     NaN
          2006     NaN
          2008     NaN
...                ...
S8TN00246 2008     NaN
S8TX00202 2008     NaN
          2010     NaN
S8TX00228 2008     NaN
S8VA00214 2004     NaN
          2006     NaN
          2008     NaN
          2012     NaN
          2014     NaN
S8VA00255 2008     NaN
          2

In [258]:
#test
agg_df.loc['S8WV00093','2008']

c8   NaN
Name: (S8WV00093, 2008), dtype: float64

In [259]:
cs9 = catsort

In [260]:
cs9 = cs9.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs9 = cs9[pd.notnull(cs9['c9'])]

In [ ]:
#cs9.groupby(['cand_id','cycle'])['c9'].sum()

In [ ]:
#cs9 = cs9.drop_duplicates()

In [261]:
cs9.set_index(['cand_id','cycle'], inplace=True)

In [262]:
agg_df = cs9.sum(level=['cand_id','cycle'])
print(agg_df)

                 c9
cand_id   cycle    
H0AK00089 2010  NaN
          2012  NaN
H0AK01038 2010  NaN
H0AL00016 2010  NaN
H0AL01030 2010  NaN
          2012  NaN
H0AL02087 2010  NaN
          2012  NaN
          2014  NaN
H0AL02103 2010  NaN
H0AL02111 2010  NaN
H0AL03184 2010  NaN
H0AL05049 2004  NaN
          2006  NaN
          2008  NaN
          2010  NaN
          2012  NaN
          2014  NaN
H0AL05155 2010  NaN
          2012  NaN
H0AL05163 2010  NaN
          2012  NaN
          2014  NaN
H0AL05171 2010  NaN
H0AL05189 2010  NaN
H0AL05197 2010  NaN
          2012  NaN
H0AL07060 2004  NaN
          2006  NaN
          2008  NaN
...              ..
S8TN00246 2008  NaN
S8TX00202 2008  NaN
          2010  NaN
S8TX00228 2008  NaN
S8VA00214 2004  NaN
          2006  NaN
          2008  NaN
          2012  NaN
          2014  NaN
S8VA00255 2008  NaN
          2010  NaN
S8VA00263 2008  NaN
          2012  NaN
          2014  NaN
S8WA00194 2006  NaN
          2008  NaN
          2010  NaN


In [263]:
cs9.head()

,,c9
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [264]:
#test
agg_df.loc['H0AL05163','2010']

c9   NaN
Name: (H0AL05163, 2010), dtype: float64

In [265]:
cs10 = catsort

In [266]:
cs10 = cs10.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs10 = cs10[pd.notnull(cs10['c10'])]

In [ ]:
#cs10.groupby(['cand_id','cycle'])['c10'].sum()

In [ ]:
#cs10 = cs10.drop_duplicates()

In [267]:
cs10.set_index(['cand_id','cycle'], inplace=True)

In [268]:
cs10.head()

,,c10
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [269]:
agg_df = cs10.sum(level=['cand_id','cycle'])
print(agg_df)

                   c10
cand_id   cycle       
H0AK00089 2010     NaN
          2012     NaN
H0AK01038 2010     NaN
H0AL00016 2010     NaN
H0AL01030 2010     NaN
          2012     NaN
H0AL02087 2010     NaN
          2012     NaN
          2014     NaN
H0AL02103 2010     NaN
H0AL02111 2010     NaN
H0AL03184 2010     NaN
H0AL05049 2004     NaN
          2006     NaN
          2008     NaN
          2010     NaN
          2012     NaN
          2014     NaN
H0AL05155 2010   610.0
          2012     NaN
H0AL05163 2010     NaN
          2012     NaN
          2014     NaN
H0AL05171 2010     NaN
H0AL05189 2010     NaN
H0AL05197 2010     NaN
          2012     NaN
H0AL07060 2004     NaN
          2006     NaN
          2008     NaN
...                ...
S8TN00246 2008     NaN
S8TX00202 2008     NaN
          2010     NaN
S8TX00228 2008     NaN
S8VA00214 2004     NaN
          2006     NaN
          2008     NaN
          2012     NaN
          2014     NaN
S8VA00255 2008     NaN
          2

In [270]:
#test
agg_df.loc['H0AL05155','2010']

c10    610.0
Name: (H0AL05155, 2010), dtype: float64

In [271]:
cs11 = catsort

In [272]:
cs11 = cs11.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c12', 'c13','total'], axis = 1)

In [273]:
cs11.head()

,cand_id,cycle,c11
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [ ]:
#cs11 = cs11[pd.notnull(cs11['c11'])]

In [ ]:
#cs11.groupby(['cand_id','cycle'])['c11'].sum()

In [ ]:
#cs11 = cs11.drop_duplicates()

In [274]:
cs11.set_index(['cand_id','cycle'], inplace=True)

In [275]:
cs11.head()

,,c11
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [276]:
agg_df = cs11.sum(level=['cand_id','cycle'])
print(agg_df)

                   c11
cand_id   cycle       
H0AK00089 2010     NaN
          2012     NaN
H0AK01038 2010     NaN
H0AL00016 2010     NaN
H0AL01030 2010     NaN
          2012     NaN
H0AL02087 2010     NaN
          2012     NaN
          2014     NaN
H0AL02103 2010     NaN
H0AL02111 2010     NaN
H0AL03184 2010     NaN
H0AL05049 2004     NaN
          2006     NaN
          2008     NaN
          2010     NaN
          2012     NaN
          2014     NaN
H0AL05155 2010     NaN
          2012     NaN
H0AL05163 2010     NaN
          2012     NaN
          2014     NaN
H0AL05171 2010     NaN
H0AL05189 2010     NaN
H0AL05197 2010   500.0
          2012     NaN
H0AL07060 2004     NaN
          2006     NaN
          2008     NaN
...                ...
S8TN00246 2008     NaN
S8TX00202 2008     NaN
          2010     NaN
S8TX00228 2008     NaN
S8VA00214 2004     NaN
          2006     NaN
          2008     NaN
          2012     NaN
          2014     NaN
S8VA00255 2008     NaN
          2

In [277]:
#test
agg_df.loc['H0AL05197','2010']

c11    500.0
Name: (H0AL05197, 2010), dtype: float64

In [278]:
cs12 = catsort

In [279]:
cs12 = cs12.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c13','total'], axis = 1)

In [280]:
cs12.head()

,cand_id,cycle,c12
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [ ]:
#cs12 = cs12[pd.notnull(cs12['c12'])]

In [ ]:
#cs12.groupby(['cand_id','cycle'])['c12'].sum()

In [ ]:
#cs12 = cs12.drop_duplicates()

In [281]:
cs12.set_index(['cand_id','cycle'], inplace=True)

In [282]:
cs12.head()

,,c12
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [283]:
agg_df = cs12.sum(level=['cand_id','cycle'])
print(agg_df)

                    c12
cand_id   cycle        
H0AK00089 2010      NaN
          2012      NaN
H0AK01038 2010      NaN
H0AL00016 2010    300.0
H0AL01030 2010      NaN
          2012      NaN
H0AL02087 2010      NaN
          2012      NaN
          2014      NaN
H0AL02103 2010      NaN
H0AL02111 2010      NaN
H0AL03184 2010      NaN
H0AL05049 2004      NaN
          2006      NaN
          2008      NaN
          2010      NaN
          2012    500.0
          2014   5000.0
H0AL05155 2010      NaN
          2012      NaN
H0AL05163 2010      NaN
          2012      NaN
          2014      NaN
H0AL05171 2010      NaN
H0AL05189 2010      NaN
H0AL05197 2010      NaN
          2012      NaN
H0AL07060 2004      NaN
          2006      NaN
          2008      NaN
...                 ...
S8TN00246 2008      NaN
S8TX00202 2008      NaN
          2010      NaN
S8TX00228 2008      NaN
S8VA00214 2004      NaN
          2006      NaN
          2008      NaN
          2012      NaN
          2014  

In [284]:
#test
agg_df.loc['H0AL05049', '2012']

c12    500.0
Name: (H0AL05049, 2012), dtype: float64

In [285]:
cs13 = catsort

In [286]:
cs13 = cs13.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12','total'], axis = 1)

In [287]:
cs13.head()

,cand_id,cycle,c13
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [ ]:
#cs13 = cs13[pd.notnull(cs13['c13'])]

In [ ]:
#cs13.groupby(['cand_id','cycle'])['c13'].sum()

In [ ]:
#cs13 = cs13.drop_duplicates()

In [288]:
cs13.set_index(['cand_id','cycle'], inplace=True)

In [289]:
cs13.head()

,,c13
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [290]:
agg_df = cs13.sum(level=['cand_id','cycle'])
print(agg_df)

                         c13
cand_id   cycle             
H0AK00089 2010     218666.06
          2012       3766.93
H0AK01038 2010     168119.58
H0AL00016 2010           NaN
H0AL01030 2010       1284.15
          2012           NaN
H0AL02087 2010    1403415.39
          2012     666262.07
          2014     541474.45
H0AL02103 2010     127314.17
H0AL02111 2010           NaN
H0AL03184 2010           NaN
H0AL05049 2004     467848.90
          2006     332285.59
          2008     454202.00
          2010      10023.55
          2012        735.00
          2014           NaN
H0AL05155 2010     518734.69
          2012       9324.78
H0AL05163 2010       1626.96
          2012       2321.95
          2014        215.00
H0AL05171 2010      18926.36
H0AL05189 2010     370262.34
H0AL05197 2010     756375.70
          2012     161419.77
H0AL07060 2004    1004122.38
          2006     405922.39
          2008     579570.41
...                      ...
S8TN00246 2008     155202.00
S8TX00202 2008

In [291]:
cs13.head()

,,c13
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [292]:
#test
agg_df.loc['S8WY00163','2008']

c13    15796.0
Name: (S8WY00163, 2008), dtype: float64

In [293]:
pr = exp

In [294]:
pr.head()

,state_abbreviation,district,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,ge_winner_indicator,total_votes,cand_id,cmte_id,exp_sum,category,cycle
0,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,C00347872,1363.96,NaN,2012
1,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,C00347872,1363.96,NaN,2012
2,AL,7,(I),D,58193.0,NaN,183408.0,NaN,W,241601,H0AL07060,C00347872,1363.96,NaN,2012
3,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,C00347872,1204248.81,NaN,2010
4,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,C00347872,1204248.81,NaN,2010


In [295]:
pr = pr.drop(['cmte_id', 'exp_sum', 'category'], axis = 1)

In [ ]:
#exp= exp[pd.notnull(cs12['c12'])]

In [296]:
pr.head()

,state_abbreviation,district,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,ge_winner_indicator,total_votes,cand_id,cycle
0,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,2012
1,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,2012
2,AL,7,(I),D,58193.0,NaN,183408.0,NaN,W,241601,H0AL07060,2012
3,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,2010
4,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,2010


In [ ]:
#cs10 = cs10.drop_duplicates()

In [297]:
pr.set_index(['cand_id','cycle'], inplace=True)

In [298]:
pr.head()

state_abbreviation district incumbent party  primary_votes  \
cand_id   cycle                                                              
H0AL07060 2012                  AL        7       (I)     D            0.0   
          2012                  AL        7       (I)   DEM        93586.0   
          2012                  AL        7       (I)     D        58193.0   
          2010                  AL        7       (I)     D            0.0   
          2010                  AL        7       (I)   DEM        93586.0   

                 runoff_votes  general_votes  ge_runoff_election_votes  \
cand_id   cycle                                                          
H0AL07060 2012            NaN       228518.0                       NaN   
          2012            NaN       133870.0                       NaN   
          2012            NaN       183408.0                       NaN   
          2010            NaN       228518.0                       NaN   
          2010            NaN       133870.0                       NaN   

                ge_winner_indicator  total_votes  
cand_id   cycle                                   
H0AL07060 2012                    W       228518  
          2012                    W       227456  
          2012                    W       241601  
          2010                    W       228518  
          2010                    W       227456

In [299]:
pr['primary_votes'].fillna(0, inplace = True)

In [300]:
pr['runoff_votes'].fillna(0, inplace = True)

In [301]:
pr['ge_runoff_election_votes'].fillna(0, inplace = True)

In [302]:
pr['general_votes'].fillna(0, inplace = True)

In [303]:
pr.head()

state_abbreviation district incumbent party  primary_votes  \
cand_id   cycle                                                              
H0AL07060 2012                  AL        7       (I)     D            0.0   
          2012                  AL        7       (I)   DEM        93586.0   
          2012                  AL        7       (I)     D        58193.0   
          2010                  AL        7       (I)     D            0.0   
          2010                  AL        7       (I)   DEM        93586.0   

                 runoff_votes  general_votes  ge_runoff_election_votes  \
cand_id   cycle                                                          
H0AL07060 2012            0.0       228518.0                       0.0   
          2012            0.0       133870.0                       0.0   
          2012            0.0       183408.0                       0.0   
          2010            0.0       228518.0                       0.0   
          2010            0.0       133870.0                       0.0   

                ge_winner_indicator  total_votes  
cand_id   cycle                                   
H0AL07060 2012                    W       228518  
          2012                    W       227456  
          2012                    W       241601  
          2010                    W       228518  
          2010                    W       227456

In [304]:
pr['primary_total'] = pr.primary_votes + pr.runoff_votes

In [305]:
pr.head()

state_abbreviation district incumbent party  primary_votes  \
cand_id   cycle                                                              
H0AL07060 2012                  AL        7       (I)     D            0.0   
          2012                  AL        7       (I)   DEM        93586.0   
          2012                  AL        7       (I)     D        58193.0   
          2010                  AL        7       (I)     D            0.0   
          2010                  AL        7       (I)   DEM        93586.0   

                 runoff_votes  general_votes  ge_runoff_election_votes  \
cand_id   cycle                                                          
H0AL07060 2012            0.0       228518.0                       0.0   
          2012            0.0       133870.0                       0.0   
          2012            0.0       183408.0                       0.0   
          2010            0.0       228518.0                       0.0   
          2010            0.0       133870.0                       0.0   

                ge_winner_indicator  total_votes  primary_total  
cand_id   cycle                                                  
H0AL07060 2012                    W       228518            0.0  
          2012                    W       227456        93586.0  
          2012                    W       241601        58193.0  
          2010                    W       228518            0.0  
          2010                    W       227456        93586.0

In [306]:
pr['general_total'] = pr.general_votes + pr.ge_runoff_election_votes

In [307]:
pr.head()

state_abbreviation district incumbent party  primary_votes  \
cand_id   cycle                                                              
H0AL07060 2012                  AL        7       (I)     D            0.0   
          2012                  AL        7       (I)   DEM        93586.0   
          2012                  AL        7       (I)     D        58193.0   
          2010                  AL        7       (I)     D            0.0   
          2010                  AL        7       (I)   DEM        93586.0   

                 runoff_votes  general_votes  ge_runoff_election_votes  \
cand_id   cycle                                                          
H0AL07060 2012            0.0       228518.0                       0.0   
          2012            0.0       133870.0                       0.0   
          2012            0.0       183408.0                       0.0   
          2010            0.0       228518.0                       0.0   
          2010            0.0       133870.0                       0.0   

                ge_winner_indicator  total_votes  primary_total  general_total  
cand_id   cycle                                                                 
H0AL07060 2012                    W       228518            0.0       228518.0  
          2012                    W       227456        93586.0       133870.0  
          2012                    W       241601        58193.0       183408.0  
          2010                    W       228518            0.0       228518.0  
          2010                    W       227456        93586.0       133870.0

In [ ]:
#pd.merge(pr,cs1, how = 'left', on = 'cand_id')

In [308]:
RS1 = pr.join(cs1)

In [310]:
RS2 = RS1.join(cs2)

In [311]:
RS3 = RS2.join(cs3)

In [ ]:
RS4 = RS3.join(cs4)

In [ ]:
#cm1 = pd.merge(cs1, cs2, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs3, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs4, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs5, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs6, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs7, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs8, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs9, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs10, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs11, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs12, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = csm1.drop(['c13'], axis = 1)

In [ ]:
#csm1 = csm1.drop_duplicates()

In [ ]:
csm1.head(200)

In [ ]:
#cs12.groupby(['cand_id','cycle'])['c12'].sum()

In [ ]:
df.head()

In [ ]:
df = election_spend

In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
df.drop('cand_name', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.head(4)

In [ ]:
df.count()

In [ ]:
df.loc[:'H6FL19038']

In [ ]:
df = df[df.total_votes != 0]

In [ ]:
df.head()

In [ ]:
Expense = df[["cand_id","cand_ici","cycle","c1","c2","c3","c4","c5","c6","c7","c8","c9","c10","c11","c12","c13"]]

In [ ]:
Other = df[["cand_id","cand_pty_affiliation","cand_office_st","cand_office","cand_office_district","cand_ici","cycle","total_votes"]]

In [ ]:
Expense.head(10)

In [ ]:
Expense_Update = Expense.drop_duplicates()

In [ ]:
Expense_Update.head()

In [ ]:
results.head()

In [ ]:
df1 = results

In [ ]:
df1.fec_id.apply(str)

In [ ]:
df2 = df1[pd.notnull(df1['fec_id'])]

In [ ]:
df2.head(20)

In [ ]:
df2['incumbent']=df2['incumbent'].fillna(0)

In [ ]:
df2['primary_votes']=df2['primary_votes'].fillna(0)

In [ ]:
df2['runoff_votes']=df2['runoff_votes'].fillna(0)

In [ ]:
df2['general_votes']=df2['general_votes'].fillna(0)

In [ ]:
df2['ge_runoff_election_votes']=df2['ge_runoff_election_votes'].fillna(0)

In [ ]:
df2.head()

In [ ]:
df2['ge_winner_indicator']=df2['ge_winner_indicator'].fillna(0)

In [ ]:
df2.head(20)

In [ ]:
df3 = df2.replace("W",1)

In [ ]:
df3.head(20)

In [ ]:
df4 = df3.replace("(I)",1)

In [ ]:
df4.head(20)

In [ ]:
results_clean = df4.drop_duplicates()

In [ ]:
results_clean.count()

In [ ]:
results_clean.drop('ge_runoff_election_votes', axis=1, inplace=True)

In [ ]:
results_clean.drop('ge_winner_indicator', axis=1, inplace=True)

In [ ]:
results_clean.drop('blank', axis=1, inplace=True)

In [ ]:
results_clean.head(20)

In [ ]:
from pandas.tools.plotting import parallel_coordinates
parallel_coordinates(data, class_column='label')

In [ ]:
from sklearn.model_selection import train_test_split as tts 

X = data[[
    '??', '??', '??', '??', '??', '??', '??'
]]
y = data['label']

print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)